In [3]:
from yolov4.tf import YOLOv4, YOLODataset, save_as_tflite

yolo = YOLOv4()

yolo.config.parse_names("test/dataset/coco.names")
yolo.config.parse_cfg("config/yolov4-tiny-relu-tpu.cfg")

yolo.make_model()
yolo.load_weights(
    "yolov4-tiny-relu.weights", weights_type="yolo"
)

dataset = YOLODataset(
    config=yolo.config,
    dataset_list="test/dataset/val2017.txt",
    image_path_prefix="/home/css-wu/CloudStation/Data/val2017",
    training=False,
)

save_as_tflite(
    model=yolo.model,
    tflite_path="yolov4-tiny-relu-int8.tflite",
    quantization="full_int8",
    dataset=dataset,
)

INFO:tensorflow:Assets written to: /tmp/tmpvn8sbmhh/assets


In [4]:
!edgetpu_compiler -sa yolov4-tiny-relu-int8.tflite

Edge TPU Compiler version 15.0.340273435

Model compiled successfully in 682 ms.

Input model: yolov4-tiny-relu-int8.tflite
Input size: 5.91MiB
Output model: yolov4-tiny-relu-int8_edgetpu.tflite
Output size: 5.98MiB
On-chip memory used for caching model parameters: 5.83MiB
On-chip memory remaining for caching model parameters: 1.78MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 2
Total number of operations: 51
Operation log: yolov4-tiny-relu-int8_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider updating your model to use only operations supported by the Edge TPU. For details, visit g.co/coral/model-reqs.
Number of operations that will run on Edge TPU: 46
Number of operations that will run on CPU: 5

Operator                       Count      Status

SPLIT                          3          Mappe

In [3]:
import cv2

from yolov4.tflite import YOLOv4

yolo = YOLOv4()

yolo.config.parse_names("/home/css-wu/CloudStation/Data/coco/classes/coco.names")
yolo.config.parse_cfg("config/yolov4-tiny-relu-tpu.cfg")

yolo.summary()

yolo.load_tflite("yolov4-tiny-relu-int8_edgetpu.tflite")

yolo.inference(
    "/home/css-wu/CloudStation/Data/cam6.mp4",
    is_image=False,
    cv_apiPreference=cv2.CAP_ANY,
    cv_frame_size=(1920, 1080),
    cv_fourcc="YUYV",
)

batch: 32
index layer No. filters  size/strd(dil)   input_shape         output_shape    1e9 flops
   0  convo_0      32      3 x 3 / 2      416 x 416 x   3 ->  208 x 208 x  32   0.075
   1  convo_1      64      3 x 3 / 2      208 x 208 x  32 ->  104 x 104 x  64   0.399
   2  convo_2      64      3 x 3 / 1      104 x 104 x  64 ->  104 x 104 x  64   0.797
   3  route_0       2,                                    ->  104 x 104 x  32
   4  convo_3      32      3 x 3 / 1      104 x 104 x  32 ->  104 x 104 x  32   0.199
   5  convo_4      32      3 x 3 / 1      104 x 104 x  32 ->  104 x 104 x  32   0.199
   6  route_1       5,  4,                                ->  104 x 104 x  64
   7  convo_5      64      1 x 1 / 1      104 x 104 x  64 ->  104 x 104 x  64   0.089
   8  route_2       2,  7,                                ->  104 x 104 x 128
   9  maxpo_0              2 x 2 / 2      104 x 104 x 128 ->   52 x  52 x 128   0.001
  10  convo_6     128      3 x 3 / 1       52 x  52 x 128 ->   52 